In [5]:
import pandas as pd
import ollama
import numpy as np
import matplotlib.pyplot as plt
from typing import List
import json
from datetime import datetime, timedelta
import random

In [9]:
def randomize(iterable: List):
    return sorted(iterable, key=lambda x: random.random())[0]

In [10]:
class ExerciseSession:
    def __init__(self):
        self.duration = randomize([30,35,40,45,50,55,60]) # seconds
        self.reps = randomize([10, 15, 20, 25, 30])
        self.errors = randomize([0, 1, 2, 3, 4, 5])
        self.exercise_type = random.choice(["pushups", "situps", "pullups", "squats","benchpress","deadlift","overheadpress"])
        self.created_at = datetime.now() - timedelta(days=randomize([1,2,3,4,5,6,7]))

class User:
    def __init__(self,first_name,fitness_goal,gender,date_of_birth):
        self.first_name = first_name
        self.fitness_goal = fitness_goal
        self.gender = gender
        self.date_of_birth = date_of_birth
        self.weight = randomize([50, 55, 60, 65, 70, 75, 80])
        self.height = randomize([150, 155, 160, 165, 170, 175, 180])
    def get_age(self):
        today = datetime.today()
        return today.year - self.date_of_birth.year - ((today.month, today.day) < (self.date_of_birth.month, self.date_of_birth.day))

In [17]:
"""
Local prompt testing environment
"""
class AIAssistant:
    def __init__(self,user:User):
        self.model_name = "mistral:latest"
        self.context = ""
        self.user = user
        greeting = "Ms." if self.user.gender == "female" else "Mr."
        self.history = [{"role": "system", 
                        "content": 
                        f"You are a fitness AI assistant for {greeting} {self.user.first_name}. {self.user.first_name} \
                        is {self.user.get_age()} years old and has the fitness goal of {self.user.fitness_goal}. \
                        {self.user.first_name} has a weight of {self.user.weight} kg and height of {self.user.height} meters. \
                        Use the information about {self.user.first_name} to provide fitness advice. "}]

    def populate_context(self, data:List[ExerciseSession])->None:
        """
        Formats exercise session data into structured context.
        """
        sessions = []
        for session in data:
            sessions.append({
                "exercise_type": session.exercise_type,
                "duration": session.duration,
                "reps": session.reps,
                "errors": session.errors,
                "created_at": session.created_at.strftime("%Y-%m-%d %H:%M:%S"),
            })
        
        self.context = json.dumps({
            "user_sessions": sessions
        }, indent=4)

    def reply(self,data:List[ExerciseSession])->str:
        # data is always non-empty, logic handled in the get request
        self.populate_context(data)
        prompt = "You are a personal trainer analyzing a user's workout history. " + \
            "The data contains parameters such as duration, reps, exercise type etc." + \
            "Provide constructive feedback on their progress, including trends, improvements, " + \
            "and areas for enhancement using the session data provided. " + \
            "Apart from the session data also use the chat history to enhance your feedback." +\
            " Use the following session data:\n" + self.context + "."
        

        self.history.append({"role": "user", "content": prompt})
        #print(f"Generating ai response...for user {self.user.id}") # useless print statement just to see if the function was entered
        response = ollama.chat(
            model=self.model_name,
            messages=self.history
        )
        
        self.history.append({"role": "assistant", "content": response["message"]["content"]})

        return response["message"]["content"]

class SingletonAIAssistant:
    _instance = None
    @staticmethod
    def get_instance(user:User):
        if SingletonAIAssistant._instance is None:
            SingletonAIAssistant._instance = AIAssistant(user=user)
        return SingletonAIAssistant._instance

In [18]:
assistant = SingletonAIAssistant.get_instance(User("John","Slim","male",datetime(1990,1,1)))

In [20]:
context = []
for i in range(10):
    context.append(ExerciseSession())

In [19]:
assistant.reply(context)

" Hello John! Based on the workout data provided, I can see that you've been consistently working out for the past week. That's fantastic! Here are some insights and suggestions to help you reach your fitness goal of slimming down more effectively:\n\n1. **Strength Training**: You've been focusing on strength training exercises like deadlift, benchpress, squats, overhead press, and pull-ups. This is great for building muscle, which will aid in weight loss as muscle burns more calories at rest compared to fat.\n\n2. **Repetitions and Errors**: The number of repetitions seems good, but there are quite a few errors during your lifts. To minimize the risk of injury and improve your form, it's essential to focus on proper technique. Consider reducing the weight slightly if needed or seeking help from a certified fitness professional.\n\n3. **Consistency**: It's great that you've been working out consistently for the past week. Continue this consistency as it plays a crucial role in achievin